# Домашнее задание №9


1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

In [1]:
# !pip install transformers

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from transformers import pipeline
from tqdm import tqdm

import pandas as pd


#### Для эксперимента возьмем модель 'SkolkovoInstitute/russian_toxicity_classifier' для классификации тональности текста

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# load tokenizer and model weights
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')


In [ ]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
model.classifier

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
# Проверка работы модели

sentiment = pipeline("text-classification", model='blanchefort/rubert-base-cased-sentiment')
sentiment("Этот ресторан плохой")

[{'label': 'NEUTRAL', 'score': 0.8267916440963745}]

In [ ]:
# Проверка работы токенайзера

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=15, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 17863, 10316,   949, 15427,   831,  2703,   102,     0,     0,
             0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
tokenizer.convert_ids_to_tokens(17863), tokenizer.convert_ids_to_tokens(15427), tokenizer.convert_ids_to_tokens(2703)

('Пример', 'ток', '##зации')

In [ ]:
# tokenizer.get_vocab()

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2022-04-21 15:41:14--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 142.250.110.139, 142.250.110.138, 142.250.110.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.110.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/aaq6f29g3no2l439vjs7hmhpa7an4itr/1650555675000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download [following]
--2022-04-21 15:41:18--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/aaq6f29g3no2l439vjs7hmhpa7an4itr/1650555675000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 74.125.206.132, 2a00:1450:400c:c04::84
Connecting to doc-14-c0-docs.googleusercontent.com (doc-14-c0-

In [219]:
!unzip data.zip

Archive:  data.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
replace val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: val.csv                 


In [220]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [222]:
df_train.head(3)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0


In [223]:
df_val.tail(3)

,id,text,class
22680,204147,"в учебники химии написано ""Пруст Жозеф Луи""\nя...",1
22681,204148,"Вот все любят,а я не люблю..как дура..говорю,ч...",0
22682,204149,уххх спасибо дорогой Леле Евгеньевне и Штепуху...,1


In [224]:
idx = np.random.randint(1,1000)
print(f'Индекс: {idx}')
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

Индекс: 190
RT @relcadoll95: Сложно грызть гранит науки?Не поддается?Ты просто не умеешь его готовить))
label is 1


[{'label': 'POSITIVE', 'score': 0.9906882047653198}]

In [225]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

- input_ids - закодированные айди токенов
- attention_mask нужна для того, чтобы PAD символы не проходили через трансформер

#### Создадим Datasets и Dataloaders

In [226]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [227]:
y_train = df_train['class']
y_val = df_val['class']

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=0)

In [228]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([128, 1, 10])


Минимально подкорректируем модель, добавив в конце функцию активации сигмоиду

In [229]:
#класс модели
class BertForSequenceClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.pretrained_model(input_ids=x, attention_mask=mask,return_dict=False)[0]  #(B, 2)
        out = self.sigm(pooled_output)
        return out

In [230]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [231]:
model = BertForSequenceClassifier().to(device)

In [232]:
model.pretrained_model.classifier

Linear(in_features=768, out_features=2, bias=True)

#### Предсказания на df_val и расчет метрик качества

In [233]:
!pip3 install torchmetrics

In [237]:
from torchmetrics import F1Score

valid_f1 = F1Score().to(device)

total_acc_val = 0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')


Val accuracy: 0.484, F1 score: 0.484


#### Дообучение модели на тренировочных данных и проверка результата на валидационных данных

In [238]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = Adam(model.pretrained_model.parameters(), lr=0.001)  

In [240]:
# train_f1 = F1Score()
# valid_f1 = F1Score()

for epoch_num in range(7):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)
        
        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        # train_f1(output, train_label)

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

        # valid_f1(output, val_label)

    # выведем метрики      
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')
    
    # train_f1.reset()
    # valid_f1.reset()

100%|██████████| 1418/1418 [12:10<00:00,  1.94it/s]


Epochs: 1 | Train Loss:  0.005         | Train Accuracy:  0.505         | Val Loss:  0.005         | Val Accuracy:  0.505


100%|██████████| 1418/1418 [12:10<00:00,  1.94it/s]


Epochs: 2 | Train Loss:  0.005         | Train Accuracy:  0.502         | Val Loss:  0.005         | Val Accuracy:  0.505


100%|██████████| 1418/1418 [12:10<00:00,  1.94it/s]


...

Epochs: 5 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [12:11<00:00,  1.94it/s]


Epochs: 6 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [12:11<00:00,  1.94it/s]


Epochs: 7 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


**Вывод:** 
дообучение модели (последнего слоя) на тренировочном датасете способствует увеличению метрики.  

**Попробуем обучить модель целиком (не только последние слои):**

In [263]:
model1 = BertForSequenceClassifier().to(device)

In [266]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = Adam(model1.parameters(), lr=0.001)  

In [271]:
sum([param.nelement() for param in model1.parameters()])

177854978

In [267]:
for epoch_num in range(5):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)
        
        output = model1(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model1(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    # выведем метрики      
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')
 

100%|██████████| 1418/1418 [11:28<00:00,  2.06it/s]


Epochs: 1 | Train Loss:  0.005         | Train Accuracy:  0.494         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [11:25<00:00,  2.07it/s]


Epochs: 2 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [11:24<00:00,  2.07it/s]


Epochs: 3 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [11:23<00:00,  2.07it/s]


Epochs: 4 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


100%|██████████| 1418/1418 [11:23<00:00,  2.07it/s]


Epochs: 5 | Train Loss:  0.005         | Train Accuracy:  0.493         | Val Loss:  0.005         | Val Accuracy:  0.495


Обучение всех параметров модели не повлияло на метрику.